In [1]:
from __future__ import division
import math
import numpy as np
import numpy.random
import scipy as sp
import scipy.stats
from scipy.optimize import minimize_scalar
from permute.utils import hypergeom_conf_interval
import itertools

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd


In [2]:
def diluted_margin_trihypergeometric2(w, l, n, N_w, N_l, N):
    pvalue = 0
    N_u = N-N_w-N_l
    for ww in range(w-l, n+1):
        tmp = 0
        for ll in range(0, ww-w+l+1):
            if ww+ll > n:
                break
            else:
                tmp += sp.misc.comb(N_l, ll)*sp.misc.comb(N_u, n-ww-ll)
        pvalue += tmp * sp.misc.comb(N_w, ww)
    return pvalue/sp.misc.comb(N, n)


def diluted_margin_trihypergeometric(w, l, n, N_w, N_l, N):
    pairs = itertools.product(range(n+1), range(n+1))
    pairs = itertools.filterfalse(lambda y: sum(y) > n, pairs)
    pairs = itertools.filterfalse(lambda y: y[0] - y[1] < (w-l), pairs)
    pvalue = 0
    N_u = N-N_w-N_l
    for p in pairs:
        if p[0] > N_w or p[1] > N_l or n-p[0]-p[1]>N_u:
            continue
        pvalue += sp.misc.comb(N_w, p[0])*sp.misc.comb(N_l, p[1])*sp.misc.comb(N_u, n-p[0]-p[1])
    return pvalue/sp.misc.comb(N, n)


def diluted_margin_trihyper_conditional(w, l, n, N_w, N_l, N):
    pvalue = 0
    delta = w-l
    n_wl = w+l
    N_u = N-N_w-N_l
    for ww in range(w, n+1):
        if ww > N_w or (n_wl - ww) > N_l or (n_wl - ww) < 0:
            continue
        pvalue += sp.misc.comb(N_w, ww)*sp.misc.comb(N_l, n_wl - ww)
    return pvalue*sp.misc.comb(N_u, n-n_wl)/sp.misc.comb(N, n)


def trihypergeometric_optim(sample, popsize, null_margin, distr = "tri-hypergeometric"):
    '''
    Wrapper function for p-value calculations using the tri-hypergeometric distribution.
    
    distr = "tri-hypergeometric" or "conditional"
    '''
    
    w = sum(sample==1)
    l = sum(sample==0)
    n = len(sample)
    u = n-w-l    

    # maximize p-value over N_wl
    if distr == "tri-hypergeometric":
        optim_fun = lambda N_w: -1*diluted_margin_trihypergeometric(w, l, n, N_w, N_w-null_margin, popsize)
        upper_WL_limit = (popsize-u+null_margin)/2
        lower_WL_limit = w
    
        res = minimize_scalar(optim_fun, 
                          bounds = [lower_WL_limit, upper_WL_limit], 
                          method = 'bounded')
    elif distr == "conditional":
        optim_fun = lambda N_w: -1*diluted_margin_trihyper_conditional(w, l, n, N_w, N_w-null_margin, popsize)
        upper_WL_limit = (popsize-u+null_margin)/2
        lower_WL_limit = w
    
        res = minimize_scalar(optim_fun, 
                          bounds = [lower_WL_limit, upper_WL_limit], 
                          method = 'bounded')
    else:
        raise ValueError("bad distr arg")
    pvalue = -1*res['fun']
    return pvalue

# Running trihypergeometric_optim(sample, popsize, null_margin=0, distr="conditional") 
# matches the plots in the other notebook.

In [3]:
# Unit tests

def test_find_pairs():
    # example: w=2, l=1, n=3
    pairs = itertools.product(range(3+1), range(3+1))
    pairs = itertools.filterfalse(lambda y: sum(y) > 3, pairs)
    pairs = itertools.filterfalse(lambda y: y[0] - y[1] < (2-1), pairs)
    expected_p = [(1, 0), (2, 0), (2, 1), (3, 0)]
    assert list(pairs)==expected_p
    
    # example: w=4, l=1, n=5
    pairs = itertools.product(range(5+1), range(5+1))
    pairs = itertools.filterfalse(lambda y: sum(y) > 5, pairs)
    pairs = itertools.filterfalse(lambda y: y[0] - y[1] < (4-1), pairs)
    expected_p = [(3, 0), (4, 0), (4, 1), (5, 0)]
    assert list(pairs)==expected_p
    
    
def test_diluted_margin_pvalue():
    # example 1: w=2, l=1, n=3, W=L=U=2
    t1 = 2*1*1/sp.misc.comb(6, 3) # w=1, l=0, u=2
    t2 = 1*1*2/sp.misc.comb(6, 3) # w=2, l=0, u=1
    t3 = 1*2*1/sp.misc.comb(6, 3) # w=2, l=1, u=0
    t4 = 0                        # w=3, l=0, u=0
    np.testing.assert_almost_equal(diluted_margin_trihypergeometric(2, 1, 3, 2, 2, 6), t1+t2+t3+t4)
    np.testing.assert_almost_equal(diluted_margin_trihypergeometric2(2, 1, 3, 2, 2, 6), t1+t2+t3+t4)
    
    # example 2: w=4, l=1, n=5, W=5, L=U=2
    t1 = sp.misc.comb(5, 3)*1*1/sp.misc.comb(9, 5) # w=3, l=0, u=2
    t2 = sp.misc.comb(5, 4)*1*2/sp.misc.comb(9, 5) # w=4, l=0, u=1
    t3 = sp.misc.comb(5, 4)*2*1/sp.misc.comb(9, 5) # w=4, l=1, u=0
    t4 = 1*1*1/sp.misc.comb(9, 5)                  # w=5, l=0, u=0
    np.testing.assert_almost_equal(diluted_margin_trihypergeometric(4, 1, 5, 5, 2, 9), t1+t2+t3+t4)
    np.testing.assert_almost_equal(diluted_margin_trihypergeometric2(4, 1, 5, 5, 2, 9), t1+t2+t3+t4)

def test_diluted_margin_trihyper_conditional():
    # example 1: w=2, l=1, n=3, W=L=U=2
    t3 = 1*2*1/sp.misc.comb(6, 3) # w=2, l=1, u=0
    t4 = 0                        # w=3, l=0, u=0
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(2, 1, 3, 2, 2, 6), t3+t4)
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(2, 1, 3, 2, 2, 6), t3+t4)

    # example 1: w=2, l=1, n=3, W=L=U=2
    hyper_prob = sp.stats.hypergeom.sf(2 - 1, 4, 2, 3) # decrement w by one to include the endpoint
    factor = sp.misc.comb(4, 3)*sp.misc.comb(2, 0)/sp.misc.comb(6, 3)
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(2, 1, 3, 2, 2, 6), hyper_prob*factor)
    
    # example 2: w=4, l=1, n=5, W=5, L=U=2
    t3 = sp.misc.comb(5, 4)*2*1/sp.misc.comb(9, 5) # w=4, l=1, u=0
    t4 = 1*1*1/sp.misc.comb(9, 5)                  # w=5, l=0, u=0
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(4, 1, 5, 5, 2, 9), t3+t4)
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(4, 1, 5, 5, 2, 9), t3+t4)

    # example 2.5: w=4, l=1, n=5, W=5, L=U=2
    hyper_prob = sp.stats.hypergeom.sf(4 - 1, 7, 5, 5)
    factor = sp.misc.comb(7, 5)*sp.misc.comb(2, 0)/sp.misc.comb(9, 5)
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(4, 1, 5, 5, 2, 9), hyper_prob*factor)
    
    # example 3: w=10, l=8, n=30, W=30, L=20, U=50
    hyper_prob = sp.stats.hypergeom.sf(10 - 1, 30 + 20, 30, 18)
    factor = sp.misc.comb(50, 18)*sp.misc.comb(50, 12)/sp.misc.comb(100, 30)
    np.testing.assert_almost_equal(diluted_margin_trihyper_conditional(10, 8, 30, 30, 20, 100), hyper_prob*factor)
    
test_find_pairs()
test_diluted_margin_pvalue()
test_diluted_margin_trihyper_conditional()

In [4]:
simTable = pd.DataFrame(columns=('vote margin', 'null margin', 'sample size', 'popsize', 'invalid_rate', \
                                 '1% rejection rate - tri', '5% rejection rate - tri', '10% rejection rate - tri', \
                                 '1% rejection rate - cond', '5% rejection rate - cond', '10% rejection rate - cond')
                       )

c_values = [0, 10]#, 50, 100]
margin = [0.01, 0.05, 0.1]
sample_rate = [0.01, 0.05, 0.1, 0.2]
invalid_rate = [0.1, 0.25, 0.5]
popsize = 1000
reps = 100
np.random.seed(837459382)

for m in margin:
    for r in invalid_rate:
        print("Margin=", m, ", invalid rate=", r)
        vote_margin = (1-r)*popsize * (m/2)
        population = [0]*(int((1-r)/2*popsize - vote_margin)) + \
                     [1]*(int((1-r)/2*popsize + vote_margin)) + \
                     [np.nan]*int(r*popsize)
        population = np.array(population)
        
        for s in sample_rate:
            for c in c_values:
                pvalues_tri = np.zeros(reps)
                pvalues_cond = np.zeros(reps)
                size = s*popsize
                
                for i in range(reps):
                    sam = np.random.choice(population, size)
                    pvalues_tri[i] = trihypergeometric_optim(sam, popsize, null_margin=c, distr = "tri-hypergeometric")
                    pvalues_cond[i] = trihypergeometric_optim(sam, popsize, null_margin=c, distr = "conditional")
                
                simTable.loc[len(simTable)] =  m, c, size, popsize, r, \
                                               np.mean(pvalues_tri <= 0.01), np.mean(pvalues_tri <= 0.05),\
                                               np.mean(pvalues_tri <= 0.1), np.mean(pvalues_cond <= 0.01), \
                                               np.mean(pvalues_cond <= 0.05), np.mean(pvalues_cond <= 0.1)

Margin= 0.01 , invalid rate= 0.1


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Margin= 0.01 , invalid rate= 0.25
Margin= 0.01 , invalid rate= 0.5
Margin= 0.05 , invalid rate= 0.1
Margin= 0.05 , invalid rate= 0.25
Margin= 0.05 , invalid rate= 0.5
Margin= 0.1 , invalid rate= 0.1
Margin= 0.1 , invalid rate= 0.25
Margin= 0.1 , invalid rate= 0.5


In [5]:
simTable

,vote margin,null margin,sample size,popsize,invalid_rate,1% rejection rate - tri,5% rejection rate - tri,10% rejection rate - tri,1% rejection rate - cond,5% rejection rate - cond,10% rejection rate - cond
0,0.01,0.0,10.0,1000.0,0.10,0.01,0.03,0.08,0.03,0.13,0.22
1,0.01,10.0,10.0,1000.0,0.10,0.01,0.01,0.04,0.01,0.06,0.06
2,0.01,0.0,50.0,1000.0,0.10,0.00,0.02,0.06,0.05,0.25,0.55
3,0.01,10.0,50.0,1000.0,0.10,0.01,0.04,0.12,0.07,0.23,0.48
4,0.01,0.0,100.0,1000.0,0.10,0.00,0.02,0.09,0.06,0.35,0.70
5,0.01,10.0,100.0,1000.0,0.10,0.00,0.03,0.07,0.05,0.34,0.66
6,0.01,0.0,200.0,1000.0,0.10,0.01,0.05,0.11,0.10,0.59,0.97
7,0.01,10.0,200.0,1000.0,0.10,0.02,0.06,0.10,0.10,0.48,0.92
8,0.01,0.0,10.0,1000.0,0.25,0.00,0.00,0.02,0.00,0.07,0.12
9,0.01,10.0,10.0,1000.0,0.25,0.00,0.01,0.09,0.01,0.15,0.23


In [6]:
simTable.to_csv("tri-hypergeometric-power-results.csv")